<a href="https://colab.research.google.com/github/sarthak-314/Book-Recommender-System/blob/master/Base%20Models/TF-IDF%20Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TF-IDF Lite**
***

In [ ]:
import pandas as pd
import os
%cd /content/drive/My\ Drive

/content/drive/My Drive


In [ ]:
DF_PATH = './Colab Notebooks/CADABRA/Recommender System/goodreads-10k'
ratings = pd.read_csv(os.path.join(DF_PATH, 'ratings.csv'))
book_tags = pd.read_csv(os.path.join(DF_PATH, 'book_tags.csv'))
tags = pd.read_csv(os.path.join(DF_PATH, 'tags.csv'))
books = pd.read_csv(os.path.join(DF_PATH, 'books.csv'))
book_tags = book_tags.merge(tags)

In [ ]:
import numpy as np
TOTAL_TAGS_COUNT = book_tags.groupby('tag_id')['count'].sum().sum()
grouped=book_tags.groupby('tag_id')['count'].sum()
def get_idf(id):
    count = grouped[id]
    return np.log(TOTAL_TAGS_COUNT/(count+25))
#TODO: Add hash table to make it fast
book_tags['idf'] = book_tags.tag_id.apply(get_idf)
book_tags.head()

,goodreads_book_id,tag_id,count,tag_name,idf
0,1,30574,167697,to-read,0.394859
1,2,30574,24549,to-read,0.394859
2,3,30574,496107,to-read,0.394859
3,5,30574,11909,to-read,0.394859
4,6,30574,298,to-read,0.394859


In [ ]:
#Helper functions
def get_tags_for_book(book_id):
    goodreads_book_id = int(books[books.book_id==book_id].goodreads_book_id)
    tags_for_book = book_tags[book_tags.goodreads_book_id==goodreads_book_id][['tag_id', 'count', 'idf']]
    return tags_for_book

def compare_books(tags_A, tags_B):
    merged = tags_A.merge(tags_B, on='tag_id')
    if merged.empty: 
        return 0
    try:
        common_tf = ((merged['count_x'] + merged['count_y']) * merged['idf_x']).sum()
        total_tf = (tags_A['count'] * tags_A['idf']).sum() + (tags_B['count']*tags_B['idf']).sum()
        similarity_index = common_tf / total_tf
        return similarity_index
    except: 
        return 0

def get_scores_with(book_id):
    tags_for_book = get_tags_for_book(book_id)
    def get_similarity_score(x):
        book_tags_x = get_tags_for_book(x.book_id)
        return compare_books(tags_for_book, book_tags_x)
    return books.apply(get_similarity_score, axis=1)


In [ ]:
books.sample(5)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
2864,2865,47558,47558,3211551,8,747541604,9.780748e+12,Edward Gorey,1963.0,The Gashlycrumb Tinies,"The Gashlycrumb Tinies (The Vinegar Works, #1)",eng,4.23,37054,38430,682,1192,1741,5141,9414,20942,https://images.gr-assets.com/books/1327933644m...,https://images.gr-assets.com/books/1327933644s...
1024,1025,7214,7214,10352,38,785263705,9.780785e+12,Donald Miller,2003.0,Blue Like Jazz: Nonreligious Thoughts on Chris...,Blue Like Jazz: Nonreligious Thoughts on Chris...,en-US,3.92,82090,83060,3546,2857,5382,16699,28410,29712,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
6224,6225,584637,584637,1086777,251,087220605X,9.780872e+12,"John Stuart Mill, George Sher",1861.0,Utilitarianism,Utilitarianism,eng,3.65,14699,15684,229,422,1440,4963,5307,3552,https://images.gr-assets.com/books/1405624617m...,https://images.gr-assets.com/books/1405624617s...
9971,9972,4984,4984,6558445,28,99282968,9.780099e+12,Kurt Vonnegut Jr.,1999.0,Bagombo Snuff Box,Bagombo Snuff Box,en-GB,3.72,7407,8597,316,78,548,2817,3430,1724,https://images.gr-assets.com/books/1327353727m...,https://images.gr-assets.com/books/1327353727s...
7473,7474,16071767,16071767,23590922,46,345539478,9.780346e+12,Karin Slaughter,2013.0,Unseen,"Unseen (Will Trent, #7)",eng,4.17,12277,16051,1226,113,392,2532,6612,6402,https://images.gr-assets.com/books/1361517437m...,https://images.gr-assets.com/books/1361517437s...


In [ ]:
def user_book_compatibility_score(user_id, book_id):
    books_rated = ratings[ratings.user_id==user_id]
    favorite_books = books_rated[books_rated.rating==5].book_id.values
    liked_books = books_rated[books_rated.rating==4].book_id.values
    hated_books = books_rated[books_rated.rating==1].book_id.values
    tags_for_book = get_tags_for_book(book_id)
    def similarity_with(books):
        if len(books) > 10:
            books = np.random.choice(books, 10)
        total_score = np.sum([compare_books(get_tags_for_book(x), tags_for_book) for x in books])/len(books)
        return total_score
    
    similarity_score = {
        'loved_books': similarity_with(favorite_books), 
        'liked_books': similarity_with(liked_books), 
        'did_not_like': similarity_with(hated_books)
    }

    return similarity_score

In [ ]:
user_book_compatibility_score(user_id=1, book_id=1)

{'did_not_like': 0.44755650315749546,
 'liked_books': 0.4944962194361791,
 'loved_books': 0.4841793114601785}

This book is closer to loved_books for the user than the did_not_like books.

This means the user will probably like this book